In [20]:
import csv
import numpy
import openpyxl
from openpyxl import Workbook
from openpyxl.writer.excel import ExcelWriter
from openpyxl.cell import get_column_letter

def open_with_csv(filename, d='\t'):
    data = []
    with open(filename, encoding='utf-8') as tsvin:
        tie_reader = csv.reader(tsvin, delimiter=d)
        for line in tie_reader:
            data.append(line)
        return data

data_from_csv = open_with_csv('data.csv')
#print(data_from_csv[0])

FIELDNAMES = ['', 'id', 'priceLabel', 'name', 'brandId', 'brandName', 'imageLink', 'desc', 'vendor',
'patterned', 'material']

DATATYPES = [('myint', 'i'), ('myid', 'i'), ('price', 'f8'), ('name', 'a200'), ('brandId', '<i8'), 
('brandName', 'a200'), ('imageUrl', '|S500'), ('description', '|S900'), ('vendor', '|S100'), 
('pattern', '|S50'), ('material', '|S50')]

def load_data(filename, d='\t'):
    my_csv = numpy.genfromtxt(filename, delimiter=d, skip_header=1, invalid_raise=False, names=FIELDNAMES,
dtype=DATATYPES)
    return my_csv 

my_csv = load_data('data.csv')

In [21]:


def number_of_records(data_sample):
    return len(data_sample)

number_of_ties = number_of_records(data_from_csv) - 1 

print(str(number_of_ties) + " ties in our data sample")

def number_of_records2(data_sample):
    return data_sample.size

number_of_ties_my_csv = number_of_records2(my_csv)
print('An alternative way of calculating ' + str(number_of_ties_my_csv) + ' ties in our data sample')

5050 ties in our data sample
An alternative way of calculating 5050 ties in our data sample


In [22]:


def calculate_sum(data_sample):
    prices = [float(row[2]) for row in data_sample[1:]]
    return sum(prices)

print('The total price of all ties combined: $' + str(calculate_sum(data_from_csv)))

def calculate_sum_concise(data_sample):
    prices = list(map(lambda x: float(x[2]), data_sample[1:]))
    return sum(prices)

print('This is a different way of calculating total price: $' + str(calculate_sum_concise(data_from_csv)))

def calc_numpy_sum(price):
    prices_in_float = [float(line) for line in price]
    total = numpy.sum(prices_in_float)
    return total

price = my_csv['priceLabel']
my_sum = calc_numpy_sum(price)
print('The (numpy) sum is: $', my_sum)


The total price of all ties combined: $702600.6900000003
This is a different way of calculating total price: $702600.6900000003
The (numpy) sum is: $ 702600.69


In [23]:


def find_average(data_sample, header=False):
    if header:
        data_sample = data_sample[1:]
    total = calculate_sum(data_sample)
    size = number_of_records(data_sample)
    average = total / size
    return average

average_price = find_average(data_from_csv, True)

print('The average price per tie is: $' + '{:03.2f}'.format(average_price))

The average price per tie is: $139.11


In [24]:


def find_min(data_sample, col):
    temp_list = []
    for row in data_sample:
        price = float(row[col])
        temp_list.append(price)
    return min(temp_list)

def find_max(data_sample, col):
    temp_list = []
    for row in data_sample:
        price = float(row[col])
        temp_list.append(price)
    return max(temp_list)

def find_max_min(data_sample, col, m='max'):
    temp_list = []
    val = 0
    for row in data_sample:
        price = float(row[col])
        temp_list.append(price)
    if m == 'max':
        val = max(temp_list)
    elif m == 'min':
        val = min(temp_list)
    else:
        pass
    return val
#print(find_max_min(data_from_csv[1:], 2, 'max'))
print('The cheapest tie in the data set is: $' + str(find_min(data_from_csv[1:], 2)))
print('The most expensive tie in the data set is: $' + str(find_max(data_from_csv[1:], 2)))

price = my_csv['priceLabel']
price_in_float = [float(x) for x in price]

numpy_max = numpy.amax(price_in_float)
numpy_min = numpy.amin(price_in_float)

print('Alternative way of calculating max ' + str(numpy_max))
print('Alternative way of calculating min ' + str(numpy_min))

The cheapest tie in the data set is: $10.0
The most expensive tie in the data set is: $711.0
Alternative way of calculating max 711.0
Alternative way of calculating min 10.0


In [25]:
def create_bool_field_from_search_term(data_sample, search_term):
    new_array = []
    new_array.append(data_sample[0].append(search_term))
    
    for row in data_sample[1:]:
        new_bool_field = False
        if search_term in row[7]:
            new_bool_field = True
            
        row.append(new_bool_field)
        new_array.append(row)
    
    return new_array
    
def filter_col_by_bool(data_sample, col):
    matches_search_term = []
    
    for item in data_sample[1:]:
        if item[col]:
            matches_search_term.append(item)
            
    return matches_search_term
    
my_new_csv = create_bool_field_from_search_term(data_from_csv, "cashmere")

print('Number of ties:', number_of_records(my_new_csv)-1)
print('Number of cashmere ties:', number_of_records(filter_col_by_bool(my_new_csv, 11)))


Number of ties: 5050
Number of cashmere ties: 56


In [26]:
def filter_col_by_string(data_sample, field, filter_condition):
    filtered_rows = []
    
    col = int(data_sample[0].index(field))
    filtered_rows.append(data_sample[0])
    
    for item in data_sample[1:]:
        if item[col] == filter_condition:
            filtered_rows.append(item)
    
    return filtered_rows

def filter_col_by_float(data_sample, field, direction, filter_condition):
    filtered_rows = []
    
    col = int(data_sample[0].index(field))
    cond = float(filter_condition)
    
    for row in data_sample[1:]:
        element = float(row[col])
        
        if direction == "<":
            if element < cond:
                filtered_rows.append(row)
        elif direction == "<=":
            if element <= cond:
                filtered_rows.append(row)
        elif direction == ">":
            if element > cond:
                filtered_rows.append(row)
        elif direction == ">=":
            if element >= cond:
                filtered_rows.append(row)
        elif direction == "==":
            if element == cond:
                filtered_rows.append(row)
        else:
            pass
    return filtered_rows

under_20_bucks = filter_col_by_float(data_from_csv, "priceLabel", '<=', 20)

silk_ties = filter_col_by_string(data_from_csv, "material", "_silk")
wool_ties = filter_col_by_string(data_from_csv, "material", "_wool")
cotton_ties = filter_col_by_string(data_from_csv, "material", "_cotton")

gucci_ties = filter_col_by_string(data_from_csv, "brandName", "Gucci")

print('Found {} ties < $20'.format(number_of_records(under_20_bucks)))
print("Found {} silk ties".format(number_of_records(silk_ties)))
print("Found {} wool ties".format(number_of_records(wool_ties)))
print("Found {} cotton ties".format(number_of_records(cotton_ties)))
print("Found {} Gucci ties".format(number_of_records(gucci_ties)))

Found 38 ties < $20
Found 3382 silk ties
Found 389 wool ties
Found 186 cotton ties
Found 171 Gucci ties


In [29]:
def save_spreadsheet(filename, data_sample):
    wb = Workbook()
    ws = wb.active
    row_index = 1
    for rows in data_sample:
        for field in rows:
            col_index = 1
            
            for field in rows:
                col_letter = get_column_letter(col_index)
                ws.cell('{}{}'.format(col_letter, row_index)).value = field
                col_index += 1
            row_index += 1
        wb.save(filename)
        
kiton_ties = filter_col_by_string(data_from_csv, "brandName", "Kiton")
save_spreadsheet("_data/s4-kiton.xlsx", kiton_ties)
    

FileNotFoundError: [Errno 2] No such file or directory: '_data/s4-kiton.xlsx'